## 1. 設定

In [ ]:
from google.colab import drive
import zipfile
import os

# Google Driveをマウント（既にマウント済みの場合はスキップ）

# マウント
drive.mount("/content/drive")

# zipファイルのパス（例：Google Drive上のアップロード先）
zip_path = "/content/drive/MyDrive/algonauts_2023_challenge_data/train_data/subj01.zip"  # 変更してください

# 展開先ディレクトリ
extract_path = "/content/data/algonauts_2023_challenge_data/train_data/"  # 変更してください

# zipファイルの確認と展開
if os.path.exists(zip_path):
    print(f"Extracting {zip_path} to {extract_path}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction complete!")
else:
    print(f"Zip file not found at: {zip_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracting /content/drive/MyDrive/algonauts_2023_challenge_data/train_data/subj01.zip to /content/data/algonauts_2023_challenge_data/train_data/
Extraction complete!


In [3]:
# =============================================================================
# 学習設定 - 必要に応じて変更してください
# =============================================================================

# 学習モード: "dummy" | "light" | "standard"
TRAIN_MODE = "light"

# 被験者ID
SUBJECT = "subj01"

# データパス（Google Drive上）
DATA_ROOT = "/mount/nfs6/visitor/nsd/algonauts_2023_challenge_data/train_data"#"/content/data/algonauts_2023_challenge_data/train_data/"

# チェックポイント保存先
CHECKPOINT_DIR = "/mount/nfs6/visitor/nsd/folder/mindeye_checkpoints"#"/content/data/mindeye_checkpoints"

# 既存の学習済みモデル（転移学習用、Noneの場合はスキップ）
PRETRAINED_CKPT = None  # 例: "/content/drive/MyDrive/train_logs/multisubject_subj01_1024hid_nolow_300ep"

# =============================================================================
# モード別設定（自動設定）
# =============================================================================
if TRAIN_MODE == "dummy":
    HIDDEN_DIM = 256
    BATCH_SIZE = 2
    NUM_EPOCHS = 1
    USE_PRIOR = False
    BLURRY_RECON = False
    DUMMY_MODE = True
elif TRAIN_MODE == "light":
    HIDDEN_DIM = 512
    BATCH_SIZE = 4
    NUM_EPOCHS = 10
    USE_PRIOR = False
    BLURRY_RECON = False
    DUMMY_MODE = False
else:  # standard
    HIDDEN_DIM = 1024
    BATCH_SIZE = 8
    NUM_EPOCHS = 50
    USE_PRIOR = True
    BLURRY_RECON = False
    DUMMY_MODE = False

print(f"Mode: {TRAIN_MODE}")
print(f"Hidden Dim: {HIDDEN_DIM}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Dummy Mode: {DUMMY_MODE}")

Mode: light
Hidden Dim: 512
Batch Size: 4
Epochs: 10
Dummy Mode: False


## 2. 環境構築

In [4]:
# GPUの確認
!nvidia-smi

Tue Jan 13 16:55:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:01:00.0 Off |                    0 |
| N/A   31C    P0             40W /  300W |      14MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
# 基本パッケージ（常に必要）
!pip install -q torch torchvision tqdm einops h5py pillow matplotlib requests numpy
!pip install -q omegaconf accelerate

# OpenAI CLIP（src/models.pyが必要）
!pip install -q git+https://github.com/openai/CLIP.git

# webdataset（src/utils.pyが必要）
!pip install -q webdataset

# OpenCLIP
!pip install -q open_clip_torch

# ⚠️ DUMMY_MODE=Trueの場合は重いライブラリをスキップ
if DUMMY_MODE:
    print("✓ DUMMY_MODE: Skipping heavy libraries (dalle2-pytorch, kornia, pytorch_lightning)")
else:
    # 本番モード用の重いパッケージ
    !pip install -q pytorch_lightning
    !pip install -q kornia
    !pip install -q dalle2-pytorch
    !pip install -q diffusers transformers

  Preparing metadata (setup.py) ... done
✓ DUMMY_MODE: Skipping heavy libraries (dalle2-pytorch, kornia, pytorch_lightning)


In [ ]:
# リポジトリのクローン（既存の場合はスキップ）
import os
if not os.path.exists("/content/MindEyeV2"):
    # TODO: 自分のフォークしたリポジトリURLに変更してください
    !git clone https://github.com/boxed-mikann/MindEyeV2.git /content/MindEyeV2
else:
    print("Repository already exists")

# パスを追加
import sys
sys.path.insert(0, "/content/MindEyeV2/mysrc")
sys.path.insert(0, "/content/MindEyeV2/src")

os.chdir("/content/MindEyeV2")
print(f"Working directory: {os.getcwd()}")

Repository already exists
Working directory: /content/MindEyeV2


In [5]:
# パスを追加
import sys ,os

sys.path.insert(0, "/home/pcnl_guest3/workspace/MindEyeV2/mysrc")
sys.path.insert(0, "/home/pcnl_guest3/workspace/MindEyeV2/src")

os.chdir("//home/pcnl_guest3/workspace/MindEyeV2")
print(f"Working directory: {os.getcwd()}")

Working directory: /home/pcnl_guest3/workspace/MindEyeV2


In [6]:
print('Pulling latest changes from MindEyeV2 repository...')
!git pull
print('Repository update complete.')

Pulling latest changes from MindEyeV2 repository...
Already up to date.
Repository update complete.


In [7]:


# データの存在確認
import os
subj_dir = os.path.join(DATA_ROOT, SUBJECT)
if os.path.exists(subj_dir):
    print(f"✓ Data found: {subj_dir}")
    !ls -la {subj_dir}
else:
    print(f"✗ Data NOT found: {subj_dir}")
    print("Please upload Algonauts2023 data to Google Drive")

✓ Data found: /mount/nfs6/visitor/nsd/algonauts_2023_challenge_data/train_data/subj01
合計 20
drwxrwxr-x  5 pcnl_guest3 visitor 4096  1月 11  2023 .
drwxrwxr-x 10 shinji      visitor 4096 12月 16 17:31 ..
drwxr-x---  2 pcnl_guest3 visitor 4096 12月 10  2022 roi_masks
drwxr-x---  3 pcnl_guest3 visitor 4096 12月 10  2022 test_split
drwxr-x---  4 pcnl_guest3 visitor 4096 10月 31  2022 training_split


## 3. データ読み込み

In [10]:
import torch
import numpy as np
from tqdm.auto import tqdm

# 環境変数でダミーモードを設定
os.environ["MINDEYE_DUMMY"] = "1" if DUMMY_MODE else "0"

# mysrcモジュールをインポート
from algonauts_dataset import AlgonautsDataset, get_dataloader, get_total_vertices
from config import print_config, DEVICE

print_config()
print(f"\nDevice: {DEVICE}")
print(f"Total vertices for {SUBJECT}: {get_total_vertices(SUBJECT)}")

ModuleNotFoundError: No module named 'numpy'

In [10]:
# =============================================================================
# 🔍 メモリデバッグ - モデル作成前
# =============================================================================
import psutil
import gc

def get_memory_info():
    """CPU/GPUメモリ情報を表示"""
    # CPUメモリ
    process = psutil.Process()
    cpu_mem_mb = process.memory_info().rss / 1024**2

    # GPUメモリ
    if torch.cuda.is_available():
        gpu_mem_mb = torch.cuda.memory_allocated() / 1024**2
        gpu_reserved_mb = torch.cuda.memory_reserved() / 1024**2
        return {
            "cpu_mb": cpu_mem_mb,
            "gpu_allocated_mb": gpu_mem_mb,
            "gpu_reserved_mb": gpu_reserved_mb,
        }
    return {"cpu_mb": cpu_mem_mb}

print("="*60)
print("📊 Memory Usage BEFORE Model Creation")
print("="*60)
mem_before = get_memory_info()
for k, v in mem_before.items():
    print(f"  {k}: {v:.2f} MB")
print("="*60)

📊 Memory Usage BEFORE Model Creation
  cpu_mb: 557.36 MB
  gpu_allocated_mb: 0.00 MB
  gpu_reserved_mb: 0.00 MB


In [11]:
# データローダー作成
train_loader = get_dataloader(
    data_root=DATA_ROOT,
    subject=SUBJECT,
    split="train",
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
)

print(f"\nDataset size: {len(train_loader.dataset)}")
print(f"Number of batches: {len(train_loader)}")

# サンプルバッチを取得
sample_batch = next(iter(train_loader))
print(f"\nSample batch:")
print(f"  fMRI shape: {sample_batch['fmri'].shape}")
print(f"  Image shape: {sample_batch['image'].shape}")

Loading fMRI data for subj01...
  LH shape: (9841, 19004), RH shape: (9841, 20544)
  Combined shape: (9841, 39548)
Found 9841 train images

Dataset size: 9841
Number of batches: 2460

Sample batch:
  fMRI shape: torch.Size([4, 39548])
  Image shape: torch.Size([4, 3, 224, 224])


## 4. モデル作成

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import pytorch_lightning as pl

: 

In [16]:
from models_algonauts import AlgonautsMindEye, create_algonauts_model
from transfer_utils import (
    load_pretrained_without_ridge,
    freeze_layers,
    get_trainable_params,
    print_parameter_summary,
)

# モデル作成
model = create_algonauts_model(
    subjects=[SUBJECT],
    hidden_dim=HIDDEN_DIM,
    seq_len=1,
    n_blocks=4,
    use_prior=USE_PRIOR,
    blurry_recon=BLURRY_RECON,
    device=DEVICE,
)

print("Model created!")
print_parameter_summary(model)

ImportError: Could not import BrainNetwork from src/models.py. Make sure src/ is in your Python path. Error: No module named 'pytorch_lightning'

In [13]:
# 転移学習（既存ckptがある場合）
if PRETRAINED_CKPT and os.path.exists(PRETRAINED_CKPT):
    print(f"Loading pretrained weights from: {PRETRAINED_CKPT}")
    loaded, missing = load_pretrained_without_ridge(
        model,
        PRETRAINED_CKPT,
        freeze_backbone=True,
        freeze_prior=True,
    )
    print(f"\nAfter transfer learning:")
    print_parameter_summary(model)
else:
    print("No pretrained checkpoint specified. Training from scratch.")
    # スクラッチ学習の場合はbackboneもfreezeしない
    pass

No pretrained checkpoint specified. Training from scratch.


In [13]:
# =============================================================================
# 🔍 メモリデバッグ - モデル作成後
# =============================================================================
print("="*60)
print("📊 Memory Usage AFTER Model Creation")
print("="*60)
mem_after = get_memory_info()
for k, v in mem_after.items():
    print(f"  {k}: {v:.2f} MB")

print("\n📈 Memory Increase:")
for k in mem_after.keys():
    increase = mem_after[k] - mem_before.get(k, 0)
    print(f"  {k}: +{increase:.2f} MB")

# モデルパラメータ数
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# 理論的なメモリ使用量（FP32想定）
theoretical_mb = total_params * 4 / 1024**2
print(f"\n🧮 Theoretical Model Size:")
print(f"  Total params: {total_params:,}")
print(f"  Trainable params: {trainable_params:,}")
print(f"  FP32 size: {theoretical_mb:.2f} MB")
print(f"  FP16 size: {theoretical_mb/2:.2f} MB")

# モデルが実際にGPU上にあるか確認
if torch.cuda.is_available():
    sample_param = next(model.parameters())
    print(f"\n🎯 Model Device: {sample_param.device}")
    print(f"   Data Type: {sample_param.dtype}")
print("="*60)

📊 Memory Usage AFTER Model Creation
  cpu_mb: 4652.56 MB
  gpu_allocated_mb: 490.02 MB
  gpu_reserved_mb: 504.00 MB

📈 Memory Increase:
  cpu_mb: +4137.39 MB
  gpu_allocated_mb: +490.02 MB
  gpu_reserved_mb: +504.00 MB

🧮 Theoretical Model Size:
  Total params: 128,452,504
  Trainable params: 128,452,504
  FP32 size: 490.01 MB
  FP16 size: 245.00 MB

🎯 Model Device: cuda:0
   Data Type: torch.float32


## 5. CLIP特徴の準備

In [11]:
# =============================================================================
# 🔍 CLIP Embedder メモリチェック
# =============================================================================
if DUMMY_MODE:
    print("="*60)
    print("📊 DummyCLIPImageEmbedder Memory Check")
    print("="*60)

    # パラメータ数
    clip_params = sum(p.numel() for p in clip_embedder.parameters())
    clip_size_mb = clip_params * 4 / 1024**2  # FP32

    print(f"  Total params: {clip_params:,}")
    print(f"  Theoretical size (FP32): {clip_size_mb:.2f} MB")

    # デバイス確認
    sample_param = next(clip_embedder.parameters())
    print(f"  Device: {sample_param.device}")
    print(f"  Dtype: {sample_param.dtype}")

    # メモリ使用量
    mem_info = get_memory_info()
    print(f"\n  Current CPU RAM: {mem_info['cpu_mb']:.2f} MB")
    if 'gpu_allocated_mb' in mem_info:
        print(f"  Current GPU RAM: {mem_info['gpu_allocated_mb']:.2f} MB")

    print("="*60)

📊 DummyCLIPImageEmbedder Memory Check


NameError: name 'clip_embedder' is not defined

In [11]:
# ダミーモードの場合はダミーCLIPを使用
if DUMMY_MODE:
    from dummy_models import DummyCLIPImageEmbedder

    clip_embedder = DummyCLIPImageEmbedder().to(DEVICE)
    print("Using DummyCLIPImageEmbedder (軽量ダミーモデル)")

else:
    # 本物のCLIPを使用（軽量版）
    # ViT-bigG-14は5GB以上でColab無料版では使えないため、軽量版を使用
    try:
        import open_clip

        # 軽量版CLIP（ViT-B/32: 約350MB）
        print("Loading OpenCLIP ViT-B-32 (lightweight version for Colab)...")
        clip_model, _, preprocess = open_clip.create_model_and_transforms(
            "ViT-B-32",  # bigG-14の代わりに軽量版を使用
            pretrained="laion2b_s34b_b79k",
        )
        clip_model = clip_model.to(DEVICE).eval()

        for param in clip_model.parameters():
            param.requires_grad = False

        print(f"OpenCLIP loaded! Model on device: {next(clip_model.parameters()).device}")

    except Exception as e:
        print(f"Failed to load OpenCLIP: {e}")
        print("Falling back to dummy mode")
        DUMMY_MODE = True
        from dummy_models import DummyCLIPImageEmbedder
        clip_embedder = DummyCLIPImageEmbedder().to(DEVICE)

Loading OpenCLIP ViT-B-32 (lightweight version for Colab)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

OpenCLIP loaded! Model on device: cuda:0


In [19]:
def get_clip_features(images):
    """画像からCLIP特徴を抽出"""
    with torch.no_grad():
        if DUMMY_MODE:
            return clip_embedder(images)
        else:
            # OpenCLIPを使用
            features = clip_model.encode_image(images)
            return features

# テスト
test_images = sample_batch['image'].to(DEVICE)
test_features = get_clip_features(test_images)
print(f"CLIP features shape: {test_features.shape}")

CLIP features shape: torch.Size([2, 512])


## 6. 学習ループ

In [20]:
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

# オプティマイザ（学習可能なパラメータのみ）
trainable_params = get_trainable_params(model, mode="all_unfrozen")
print(f"Trainable parameters: {sum(p.numel() for p in trainable_params):,}")

optimizer = AdamW(trainable_params, lr=3e-4, weight_decay=1e-2)

# スケジューラ
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = OneCycleLR(
    optimizer,
    max_lr=3e-4,
    total_steps=total_steps,
    pct_start=0.1,
)

Trainable parameters: 128,452,504


In [ ]:
def soft_clip_loss(preds, targets, temp=0.006):
    """
    Soft CLIP contrastive loss (元のutils.pyと同じロジック)
    
    Args:
        preds: (batch, emb_dim) - モデル予測（brain → CLIP）
        targets: (batch, emb_dim) - 実際のCLIP特徴
        temp: temperature parameter
    """
    # preds/targetsは2次元 (batch, emb_dim) を想定
    # もし3次元なら最後の次元以外をflatten
    if preds.dim() == 3:
        preds = preds.reshape(preds.shape[0], -1)
    if targets.dim() == 3:
        targets = targets.reshape(targets.shape[0], -1)
    
    # 次元が一致しない場合はpredsを切り詰める（緊急対応）
    if preds.shape[1] != targets.shape[1]:
        print(f"Warning: dimension mismatch - preds:{preds.shape[1]}, targets:{targets.shape[1]}")
        # CLIPの次元に合わせて線形変換（学習可能にする場合は要検討）
        if not hasattr(soft_clip_loss, 'dim_adapter'):
            soft_clip_loss.dim_adapter = nn.Linear(preds.shape[1], targets.shape[1]).to(preds.device)
        preds = soft_clip_loss.dim_adapter(preds)
    
    # 元のutils.pyと同じロジック
    clip_clip = (targets @ targets.T) / temp
    brain_clip = (preds @ targets.T) / temp
    
    loss1 = -(brain_clip.log_softmax(-1) * clip_clip.softmax(-1)).sum(-1).mean()
    loss2 = -(brain_clip.T.log_softmax(-1) * clip_clip.softmax(-1)).sum(-1).mean()
    
    return (loss1 + loss2) / 2

def train_step(batch):
    """1バッチの学習ステップ"""
    model.train()

    # データ取得
    fmri = batch['fmri'].to(DEVICE)
    images = batch['image'].to(DEVICE)

    # CLIP特徴を取得（ターゲット）
    with torch.no_grad():
        clip_target = get_clip_features(images)  # (batch, emb_dim)

    # Forward
    backbone, clip_voxels, blurry = model(fmri)

    # 形状をデバッグ出力（最初のバッチのみ）
    if not hasattr(train_step, 'debug_printed'):
        print(f"Debug shapes:")
        print(f"  fmri: {fmri.shape}")
        print(f"  backbone: {backbone.shape}")
        print(f"  clip_voxels: {clip_voxels.shape}")
        print(f"  clip_target: {clip_target.shape}")
        train_step.debug_printed = True

    # clip_voxelsをflattenして使う
    # clip_voxels: (batch, seq_len, clip_emb_dim) -> (batch, seq_len * clip_emb_dim)
    clip_voxels_flat = clip_voxels.reshape(clip_voxels.shape[0], -1)
    clip_target_flat = clip_target.reshape(clip_target.shape[0], -1)

    # Loss計算
    loss = soft_clip_loss(clip_voxels_flat, clip_target_flat)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(trainable_params, 1.0)
    optimizer.step()
    scheduler.step()

    return loss.item()

In [22]:
# メモリ確認
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB allocated")
    print(f"GPU Memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB reserved")

GPU Memory: 1.06 GB allocated
GPU Memory: 1.10 GB reserved


In [23]:
# 学習ループ
print(f"\n{'='*60}")
print(f"Starting training: {NUM_EPOCHS} epochs, {len(train_loader)} batches/epoch")
print(f"{'='*60}\n")

losses = []

for epoch in range(NUM_EPOCHS):
    epoch_losses = []

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for batch in pbar:
        loss = train_step(batch)
        epoch_losses.append(loss)
        pbar.set_postfix({"loss": f"{loss:.4f}"})

    avg_loss = np.mean(epoch_losses)
    losses.extend(epoch_losses)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Average Loss: {avg_loss:.4f}")

    # メモリ確認
    if torch.cuda.is_available():
        print(f"  GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

print(f"\n{'='*60}")
print("Training complete!")
print(f"{'='*60}")


Starting training: 1 epochs, 4920 batches/epoch



Epoch 1/1:   0%|          | 0/4920 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x425984 and 512x2)

In [ ]:
# 損失の可視化
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.grid(True)
plt.show()

## 7. チェックポイント保存

In [ ]:
from transfer_utils import save_checkpoint

# 保存先ディレクトリ作成
save_dir = os.path.join(CHECKPOINT_DIR, f"algonauts_{SUBJECT}_{TRAIN_MODE}")
os.makedirs(save_dir, exist_ok=True)

# チェックポイント保存
save_checkpoint(
    model=model,
    optimizer=optimizer,
    epoch=NUM_EPOCHS,
    save_path=os.path.join(save_dir, "last.pth"),
    extra_info={
        "train_mode": TRAIN_MODE,
        "subject": SUBJECT,
        "hidden_dim": HIDDEN_DIM,
        "final_loss": losses[-1] if losses else None,
    },
)

print(f"\nCheckpoint saved to: {save_dir}")

## 8. 簡易検証

In [ ]:
# 推論テスト
model.eval()

with torch.no_grad():
    test_batch = next(iter(train_loader))
    test_fmri = test_batch['fmri'].to(DEVICE)
    test_images = test_batch['image'].to(DEVICE)

    # fMRI → CLIP tokens
    backbone, clip_voxels, blurry = model(test_fmri)

    # 実際のCLIP特徴
    clip_target = get_clip_features(test_images)
    if clip_target.dim() == 2:
        clip_target = clip_target.unsqueeze(1)

    # コサイン類似度
    pred_flat = F.normalize(clip_voxels.view(clip_voxels.shape[0], -1), dim=-1)
    target_flat = F.normalize(clip_target.view(clip_target.shape[0], -1), dim=-1)

    similarity = (pred_flat * target_flat).sum(dim=-1).mean()

    print(f"\nInference test:")
    print(f"  Input fMRI shape: {test_fmri.shape}")
    print(f"  Output CLIP shape: {clip_voxels.shape}")
    print(f"  Average cosine similarity: {similarity.item():.4f}")

In [ ]:
# 入力画像の可視化
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

def denormalize(tensor):
    """ImageNet正規化を元に戻す"""
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(tensor.device)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(tensor.device)
    return tensor * std + mean

# サンプル画像を表示
sample_images = denormalize(test_images[:4])
grid = make_grid(sample_images, nrow=4).cpu().permute(1, 2, 0).numpy()
grid = np.clip(grid, 0, 1)

plt.figure(figsize=(12, 3))
plt.imshow(grid)
plt.title("Sample Training Images")
plt.axis("off")
plt.show()

## 次のステップ

1. **ダミーモードで動作確認** → エラーなく完了すればOK
2. **軽量モード（light）で実学習** → T4で数時間
3. **標準モード（standard）で本格学習** → Pro or 研究室PC
4. **推論ノートブック** → `mindeye_inference_colab.ipynb` で画像再構成